# Sample rental database

In [83]:
# Importamos el módulo sqllite
import sqlite3
import numpy as np
import pandas as pd

In [84]:
conn = sqlite3.connect('sample_rental.db')
cursor = conn.cursor()
# with open('sqlite_rental_schema.sql', 'r') as q:
#     sql = q.read()
#     cursor.executescript(sql)

In [12]:
# with open('sqlite_rental_insert.sql', 'r') as q:
#     sql = q.read()
#     cursor.executescript(sql)

In [85]:
conn = sqlite3.connect('sample_rental.db')

tables = pd.read_sql(""" select *
                        from sqlite_master 
                        where type='table'
                        order by name;
                        """, conn)

In [86]:
pd.set_option("display.max_columns", None)

In [87]:
tables

,type,name,tbl_name,rootpage,sql
0,table,actor,actor,2,CREATE TABLE actor (\n actor_id numeric NOT N...
1,table,address,address,10,CREATE TABLE address (\n address_id int NOT N...
2,table,category,category,15,CREATE TABLE category (\n category_id SMALLIN...
3,table,city,city,7,"CREATE TABLE city (\n city_id int NOT NULL,\n..."
4,table,country,country,5,CREATE TABLE country (\n country_id SMALLINT ...
5,table,customer,customer,19,CREATE TABLE customer (\n customer_id INT NOT...
6,table,film,film,24,"CREATE TABLE film (\n film_id int NOT NULL,\n..."
7,table,film_actor,film_actor,28,CREATE TABLE film_actor (\n actor_id INT NOT ...
8,table,film_category,film_category,33,CREATE TABLE film_category (\n film_id INT NO...
9,table,film_text,film_text,37,CREATE TABLE film_text (\n film_id SMALLINT N...


1) Buscamos actores con mismo nombre y apellido con diferente actor_id

In [88]:
q = """
    SELECT actor_id, first_name, last_name
    FROM actor
    WHERE (first_name, last_name) in (
        SELECT first_name, last_name
        FROM actor
        GROUP BY first_name, last_name
        HAVING COUNT(actor_id) > 1
        )
"""
df = pd.read_sql(q, conn)
df

,actor_id,first_name,last_name
0,101,SUSAN,DAVIS
1,110,SUSAN,DAVIS


* 2) Selecciona actores por cada película

In [8]:
q = """
    SELECT title, first_name || " " || last_name as full_name
    FROM film
    INNER JOIN film_actor 
    ON film.film_id = film_actor.film_id
    INNER JOIN actor
    ON actor.actor_id = film_actor.actor_id
    GROUP BY title, full_name
    ORDER BY title, full_name
"""
df = pd.read_sql(q, conn)
df

,title,full_name
0,ACADEMY DINOSAUR,CHRISTIAN GABLE
1,ACADEMY DINOSAUR,JOHNNY CAGE
2,ACADEMY DINOSAUR,LUCILLE TRACY
3,ACADEMY DINOSAUR,MARY KEITEL
4,ACADEMY DINOSAUR,MENA TEMPLE
...,...,...
5457,ZOOLANDER FICTION,PENELOPE CRONYN
5458,ZOOLANDER FICTION,WHOOPI HURT
5459,ZORRO ARK,IAN TANDY
5460,ZORRO ARK,LISA MONROE


* 3) Contar actores por cada película

**subquery** :
```
q = """
    SELECT actor_id, first_name, last_name
    FROM actor
    WHERE (first_name, last_name) in (
        SELECT first_name, last_name
        FROM actor
        GROUP BY first_name, last_name
        HAVING COUNT(actor_id) > 1
        )
"""
df = pd.read_sql(q, conn)
df

---

WITH tabla1 AS 
    (SELECT ...FROM ...WHERE ...GROUP BY)

tabla2 AS 
    (SELECT ... FROM ...WHERE ...GROUP BY)


SELECT tabla1.col1, tabla2.col3 FROM

```

In [10]:
q = """
    WITH t1 AS (
      SELECT title, first_name || " " || last_name as full_name
        FROM film
        INNER JOIN film_actor 
        ON film.film_id = film_actor.film_id
        INNER JOIN actor
        ON actor.actor_id = film_actor.actor_id
        GROUP BY title, full_name
    )      
    SELECT
        title
        ,count(full_name) as count_of_actors
    FROM t1
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 10
"""
df = pd.read_sql(q, conn)
df

,title,count_of_actors
0,LAMBS CINCINATTI,15
1,RANDOM GO,13
2,MUMMY CREATURES,13
3,DRACULA CRYSTAL,13
4,CRAZY HOME,13
5,CHITTY LOCK,13
6,BOONDOCK BALLROOM,13
7,TITANIC BOONDOCK,12
8,SKY MIRACLE,12
9,LONELY ELEPHANT,12


df[1000 rows, 10 cols]

# Crear una nueva columna al dataframe y asignamos un None
df['new_col'] = None


def crear_rank(row):
    df.index + 1

df['new_col'] = df.apply(df.index + 1, df.shape[0]

* 4. Crear la top 10 de los actores donde ordenados por película de mayor a menor

In [26]:
q = """
WITH t1 AS (
      SELECT first_name || " " || last_name as full_name,
        count(film.film_id) as movie_count,
        dense_rank() OVER (ORDER BY COUNT(film.film_id) desc) as d_rank
        FROM film
        INNER JOIN film_actor 
        ON film.film_id = film_actor.film_id
        INNER JOIN actor
        ON actor.actor_id = film_actor.actor_id
        GROUP BY full_name
        ORDER BY 2 DESC
      )
  
  SELECT 
    full_name as nombre
    ,movie_count as conteo
    ,d_rank as rank 
  FROM t1
  WHERE d_rank <= 10;
    """
df = pd.read_sql(q, conn)
df

,nombre,conteo,rank
0,SUSAN DAVIS,54,1
1,GINA DEGENERES,42,2
2,WALTER TORN,41,3
3,MARY KEITEL,40,4
4,MATTHEW CARREY,39,5
5,SANDRA KILMER,37,6
6,SCARLETT DAMON,36,7
7,VIVIEN BASINGER,35,8
8,VAL BOLGER,35,8
9,UMA WOOD,35,8


In [28]:
df.columns

Index(['nombre', 'conteo', 'rank'], dtype='object')

In [41]:
df[(df['rank'] > 5)]

,nombre,conteo,rank
5,SANDRA KILMER,37,6
6,SCARLETT DAMON,36,7
7,VIVIEN BASINGER,35,8
8,VAL BOLGER,35,8
9,UMA WOOD,35,8
10,HENRY BERRY,35,8
11,GROUCHO DUNST,35,8
12,ANGELA WITHERSPOON,35,8
13,WARREN NOLTE,34,9
14,SIDNEY CROWE,34,9


In [29]:
df_new = df[['nombre','rank']]

,nombre,rank
0,SUSAN DAVIS,1
1,GINA DEGENERES,2
2,WALTER TORN,3
3,MARY KEITEL,4
4,MATTHEW CARREY,5
5,SANDRA KILMER,6
6,SCARLETT DAMON,7
7,VIVIEN BASINGER,8
8,VAL BOLGER,8
9,UMA WOOD,8


* 5. Mostrar el número de veces que cada película de disponible haya pasado en estado alquiler, y mostrar su precio.

In [42]:
q = '''
    SELECT * FROM payment
    WHERE payment.rental_id is NULL
'''
df = pd.read_sql(q, conn)
df

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update
0,424,16,1,None,1.99,2005-06-18 04:56:12.000,2023-05-27 11:20:39
1,7011,259,2,None,1.99,2005-08-23 06:13:16.000,2023-05-27 11:21:07
2,10840,401,1,None,0.99,2005-07-12 06:26:10.000,2023-05-27 11:21:23
3,14675,546,1,None,3.99,2005-07-30 21:16:20.000,2023-05-27 11:21:40
4,15458,577,2,None,0.99,2005-05-27 00:46:39.000,2023-05-27 11:21:45


In [76]:
q = '''
    SELECT * FROM film
    ORDER BY rental_duration ASC
'''
df = pd.read_sql(q, conn)
df

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2023-05-27 11:18:28
1,6,AGENT TRUMAN,A Intrepid Panorama of a Robot And a Boy who m...,2006,1,None,3,2.99,169,17.99,PG,Deleted Scenes,2023-05-27 11:18:28
2,9,ALABAMA DEVIL,A Thoughtful Panorama of a Database Administra...,2006,1,None,3,2.99,114,21.99,PG-13,"Trailers,Deleted Scenes",2023-05-27 11:18:28
3,17,ALONE TRIP,A Fast-Paced Character Study of a Composer And...,2006,1,None,3,0.99,82,14.99,R,"Trailers,Behind the Scenes",2023-05-27 11:18:28
4,21,AMERICAN CIRCUS,A Insightful Drama of a Girl And a Astronaut w...,2006,1,None,3,4.99,129,17.99,R,"Commentaries,Behind the Scenes",2023-05-27 11:18:28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,962,WASTELAND DIVINE,A Fanciful Story of a Database Administrator A...,2006,1,None,7,2.99,85,18.99,PG,"Trailers,Deleted Scenes,Behind the Scenes",2023-05-27 11:18:32
996,970,WESTWARD SEABISCUIT,A Lacklusture Tale of a Butler And a Husband w...,2006,1,None,7,0.99,52,11.99,NC-17,"Commentaries,Deleted Scenes",2023-05-27 11:18:32
997,981,WOLVES DESIRE,A Fast-Paced Drama of a Squirrel And a Robot w...,2006,1,None,7,0.99,55,13.99,NC-17,Behind the Scenes,2023-05-27 11:18:32
998,983,WON DARES,A Unbelieveable Documentary of a Teacher And a...,2006,1,None,7,2.99,105,18.99,PG,Behind the Scenes,2023-05-27 11:18:32


In [45]:
q = '''
    SELECT * FROM rental
    WHERE rental.customer_id = '16'
'''
df = pd.read_sql(q, conn)
df

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,335,2005-05-27 03:07:10.000,1411,16,2005-06-05 00:15:10.000,2,2023-05-27 11:19:23
1,593,2005-05-28 13:33:23.000,1810,16,2005-05-30 17:10:23.000,2,2023-05-27 11:19:25
2,887,2005-05-30 07:10:00.000,2727,16,2005-06-01 06:48:00.000,2,2023-05-27 11:19:26
3,1017,2005-05-31 02:53:36.000,1242,16,2005-06-03 05:04:36.000,1,2023-05-27 11:19:27
4,1934,2005-06-17 07:04:57.000,4000,16,2005-06-25 12:21:57.000,2,2023-05-27 11:19:31
5,1944,2005-06-17 07:50:53.000,318,16,2005-06-23 02:52:53.000,2,2023-05-27 11:19:31
6,2960,2005-06-20 07:10:09.000,3970,16,2005-06-26 08:14:09.000,2,2023-05-27 11:19:36
7,3348,2005-06-21 11:16:42.000,2945,16,2005-06-27 13:50:42.000,2,2023-05-27 11:19:37
8,3548,2005-07-06 02:23:39.000,4498,16,2005-07-08 07:53:39.000,2,2023-05-27 11:19:38
9,4219,2005-07-07 12:11:22.000,2800,16,2005-07-11 11:05:22.000,1,2023-05-27 11:19:42


In [43]:
q = '''
    SELECT * FROM payment
    WHERE payment.rental_id is NOT NULL
'''
df = pd.read_sql(q, conn)
df

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update
0,1,1,1,76,2.99,2005-05-25 11:30:37.000,2023-05-27 11:20:38
1,2,1,1,573,0.99,2005-05-28 10:35:23.000,2023-05-27 11:20:38
2,3,1,1,1185,5.99,2005-06-15 00:54:12.000,2023-05-27 11:20:38
3,4,1,2,1422,0.99,2005-06-15 18:02:53.000,2023-05-27 11:20:38
4,5,1,2,1476,9.99,2005-06-15 21:08:46.000,2023-05-27 11:20:38
...,...,...,...,...,...,...,...
16039,16045,599,1,14599,4.99,2005-08-21 17:43:42.000,2023-05-27 11:21:49
16040,16046,599,1,14719,1.99,2005-08-21 21:41:57.000,2023-05-27 11:21:49
16041,16047,599,2,15590,8.99,2005-08-23 06:09:44.000,2023-05-27 11:21:49
16042,16048,599,2,15719,2.99,2005-08-23 11:08:46.000,2023-05-27 11:21:49


* 6. Mostrar el número de clientes que no han devuelto más de 1 DVD?

In [79]:
q = '''
    SELECT 
        c.first_name
        ,c.last_name
        ,c.email
        ,count(r.rental_id) as no_devueltos
        ,r.rental_date
        ,c.active
        ,p.amount
        ,julianday(r.return_date) - julianday(r.rental_date) as duration
    FROM customer as c
    INNER JOIN rental as r
    ON c.customer_id = r.customer_id
    INNER JOIN payment as p
    ON p.customer_id = c.customer_id
    WHERE r.return_date NOT NULL and c.active = 1
    GROUP BY r.customer_id
    HAVING no_devueltos >= 1
    ORDER BY 3 DESC
'''
df = pd.read_sql(q, conn)
df

,first_name,last_name,email,no_devueltos,rental_date,active,amount,duration
0,ZACHARY,HITE,ZACHARY.HITE@sakilacustomer.org,930,2005-05-25 21:46:54.000,1,3.99,5.969444
1,YVONNE,WATKINS,YVONNE.WATKINS@sakilacustomer.org,441,2005-05-25 05:12:29.000,1,5.99,7.802778
2,YOLANDA,WEAVER,YOLANDA.WEAVER@sakilacustomer.org,702,2005-05-27 16:22:10.000,1,4.99,8.866667
3,WILMA,RICHARDS,WILMA.RICHARDS@sakilacustomer.org,400,2005-06-15 13:17:01.000,1,0.99,4.128472
4,WILLIE,MARKHAM,WILLIE.MARKHAM@sakilacustomer.org,600,2005-05-26 19:21:44.000,1,8.99,8.789583
...,...,...,...,...,...,...,...,...
579,ALAN,KAHN,ALAN.KAHN@sakilacustomer.org,676,2005-05-31 00:16:57.000,1,4.99,1.934028
580,AGNES,BISHOP,AGNES.BISHOP@sakilacustomer.org,529,2005-05-29 22:14:55.000,1,2.99,0.797222
581,ADRIAN,CLARY,ADRIAN.CLARY@sakilacustomer.org,342,2005-05-27 17:47:22.000,1,5.99,8.857639
582,ADAM,GOOCH,ADAM.GOOCH@sakilacustomer.org,484,2005-05-30 14:49:34.000,1,0.99,2.174306


1. **¿Tenemos actores en la tabla de actores que comparten el nombre completo y, en caso afirmativo, muestran esos nombres compartidos?**


Para responder a esa pregunta, primero necesitamos saber si tenemos actores en la tabla de actores que comparten sus nombres completos. Entonces comenzar contando la cantidad de actores que comparten sus nombres completos.



In [89]:
q = '''
    SELECT 
        COUNT(DISTINCT first_name || last_name)
    FROM actor;
'''
df = pd.read_sql(q, conn)
df

,COUNT(DISTINCT first_name || last_name)
0,199


In [90]:
q = """
    SELECT
	DISTINCT a1.first_name, a1.last_name
FROM actor a1 JOIN actor a2
ON a1.actor_id <> a2.actor_id AND a1.first_name = a2.first_name AND a1.last_name = a2.last_name;
"""
df = pd.read_sql(q, conn)
df

,first_name,last_name
0,SUSAN,DAVIS


2. **Muestra los nombres de los clientes que comparten la misma dirección (por ejemplo, esposo y esposa).**

Para responder a esta pregunta, debemos buscar más de un cliente que tenga el mismo address_id pero diferentes customer_ids. Eso significa que haremos JOIN a la tabla de clientes para sí mismo (*autounión*).

In [ ]:
q = """
    SELECT c1.first_name, c1.last_name, c2.first_name, c2.last_name
FROM customer c1 JOIN customer c2

"""
df = pd.read_sql(q, conn)
df

3. **Muestra el monto total pagado por todos los clientes en la tabla de pagos.**

Utilizar aquí la función agregada **SUM()** en la columna de monto de la tabla de pagos.



4. **Muestre el monto total pagado por cada cliente en la tabla de pagos.**


5. **¿Cuál es el pago total más alto realizado?**

In [92]:
q = '''SELECT SUM(amount) as monto_total
FROM payment
GROUP BY customer_id
ORDER BY SUM(amount) DESC
LIMIT 1
    '''
df = pd.read_sql(q, conn)
df

,monto_total
0,221.55
